In [255]:
import numpy
import scipy.io.wavfile
from scipy.fftpack import dct
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from matplotlib import cm
from pylab import *
import os
import pandas as pd
import gc
from scipy.signal import blackman
import csv
from scipy.io import wavfile
import wave
from struct import pack
import plotly,plotly as py
#from dtw import dtw
from numpy.linalg import norm
from datetime import datetime
from datetime import timedelta
import ipywidgets as widgets
#from numba import vectorize

gc.enable()

data_path = "data/"
date = "20181205"
stereo_path = "stereo/"
vector_path = "vector/"
visual_path = "visual/"

video_start = '20190222' #YYYYMMDD
video_end = '20190222' #YYYYMMDD
temp_video_start = video_start[0:4] + "/" + video_start[4:6] + "/" +video_start[6:8]
temp_video_end = video_end[0:4] + "/" + video_end[4:6] + "/" +video_end[6:8]
print temp_video_start
print temp_video_end

video_startdatte = datetime.strptime(temp_video_start,'%Y/%m/%d')
video_enddate = datetime.strptime(temp_video_end,'%Y/%m/%d')

directory_list = ['cloth','metal','paper','plastic','porcelain','wood']

for mat in directory_list:
    if not os.path.exists(visual_path+mat):
        os.makedirs(visual_path+mat)
        
#material = "porcelain/"

2019/02/22
2019/02/22


In [262]:
session_list = []
with open('data/capture_time.csv') as csvfile: #read session data from capture_time.csv
    readCSV = csv.reader(csvfile, delimiter='-')
    line_no = 0
    print "The data sessions taken during the start and end dates"
    for row in readCSV:
        if row[0] != '' and row[1] != '': #Check if both start and end times of each row exists
            starttime = datetime.strptime(row[0],'%Y/%m/%d %H:%M:%S.%f')
            endtime = datetime.strptime(row[1],'%Y/%m/%d %H:%M:%S.%f')
            #print starttime.date()
            if starttime.date() == video_startdatte.date() and endtime.date() == video_enddate.date(): #Check if sessions available which matches the startdate and enddate
                line_no = line_no + 1
                print "(", line_no, ")", "\t", starttime, "-", endtime
                session_list.append(row[0] + "-" + row[1])

print "Please input the session number. Confirm the start and end times using the video."
session = raw_input("Session ") 
print "You have chosen the session ", session

The data sessions taken during the start and end dates
( 1 ) 	2019-02-22 12:08:45.886000 - 2019-02-22 12:10:16.645000
( 2 ) 	2019-02-22 12:19:02.759000 - 2019-02-22 12:20:41.306000
( 3 ) 	2019-02-22 12:21:37.795000 - 2019-02-22 12:23:12.849000
( 4 ) 	2019-02-22 12:25:24.705000 - 2019-02-22 12:27:04.979000
( 5 ) 	2019-02-22 12:27:45.073000 - 2019-02-22 12:29:21.248000
( 6 ) 	2019-02-22 12:30:13.576000 - 2019-02-22 12:31:51.139000
( 7 ) 	2019-02-22 12:35:10.525000 - 2019-02-22 12:36:47.742000
( 8 ) 	2019-02-22 12:38:44.271000 - 2019-02-22 12:40:34.994000
( 9 ) 	2019-02-22 12:41:37.059000 - 2019-02-22 12:43:13.028000
Please input the session number. Confirm the start and end times using the video.
Session 
You have chosen the session  


In [263]:
session_list

['2019/02/22 12:08:45.886-2019/02/22 12:10:16.645',
 '2019/02/22 12:19:02.759-2019/02/22 12:20:41.306',
 '2019/02/22 12:21:37.795-2019/02/22 12:23:12.849',
 '2019/02/22 12:25:24.705-2019/02/22 12:27:04.979',
 '2019/02/22 12:27:45.073-2019/02/22 12:29:21.248',
 '2019/02/22 12:30:13.576-2019/02/22 12:31:51.139',
 '2019/02/22 12:35:10.525-2019/02/22 12:36:47.742',
 '2019/02/22 12:38:44.271-2019/02/22 12:40:34.994',
 '2019/02/22 12:41:37.059-2019/02/22 12:43:13.028']

In [264]:
k=0
for itr in session_list:
    session_starttime = datetime.strptime(itr[0:23],'%Y/%m/%d %H:%M:%S.%f')
    session_endtime = datetime.strptime(itr[24:47],'%Y/%m/%d %H:%M:%S.%f')
    session_list[k] = str(session_starttime) + "-" + str(session_endtime)
    k += 1

In [265]:
#creating labels for each session
dic={}
dic["bed"] = [session_list[0],session_list[1],session_list[2]]
dic["kit"] = [session_list[3],session_list[4],session_list[5]]
dic["bath"] = [session_list[6],session_list[7],session_list[8]]

In [266]:
features = pd.DataFrame(columns=['label','feature'])
bookmark=0
for label in dic:
    for sess in dic[label]:
        my_data = genfromtxt("vector/"+sess+'.csv', delimiter=',')
        features.loc[bookmark]=label,my_data
        bookmark+=1

In [268]:
features

,label,feature
0,bath,"[[-6.922662233858751, -23.7087022722078, 4.713..."
1,bath,"[[-3.4460855511189727, -25.42134933017988, 7.4..."
2,bath,"[[0.16496102169309998, -22.37723438881532, 8.8..."
3,bed,"[[-48.45100711287336, -41.53818909791094, 6.38..."
4,bed,"[[-5.038652178242667, -21.816600466416872, 7.0..."
5,bed,"[[-13.010142584152945, -30.177122297832643, 7...."
6,kit,"[[-8.220361991790492, -23.03664237913704, 6.93..."
7,kit,"[[-15.115335858638916, -29.5165055169947, 4.05..."
8,kit,"[[-55.6165207093193, -43.26069421587071, 6.513..."


In [203]:
features['feature'].values

array([array([[-6.92266223e+00, -2.37087023e+01,  4.71381772e+00],
       [-8.03347803e+00, -2.37441165e+01,  3.34659182e+00],
       [-3.50615346e+00, -1.67952795e+01,  3.82607465e+00],
       [ 2.40374433e+00, -1.62445405e+01,  7.42477465e+00],
       [-5.12568326e+00, -1.61201138e+01,  2.73589574e+00],
       [-4.65226324e+00, -1.59862674e+01,  2.85080023e+00],
       [-9.25020332e+00, -1.93324169e+01, -4.03538751e-01],
       [-8.29681410e+00, -1.90429162e+01,  4.62862546e-01],
       [-3.21117454e+00, -1.96841230e+01,  5.66075405e-01],
       [-4.33178172e+00, -2.48035654e+01,  3.00889523e+00],
       [-5.53321527e+00, -3.10769064e+01,  1.37116994e+00],
       [ 1.49324118e+01, -2.61208174e+01,  2.06773833e+00],
       [ 4.25206201e+01, -1.17867382e+01,  2.31231646e-02],
       [ 6.24249873e+01,  1.05225006e+01, -8.01511704e+00],
       [ 1.71814287e+01,  1.91621151e+01, -2.84663740e+00],
       [-1.14832514e+01,  2.32617388e+01,  6.52901901e-01],
       [-3.37727354e+01,  2.44125

In [206]:
numpy.concatenate( features['feature'].values, axis=0 )

array([[-6.92266223e+00, -2.37087023e+01,  4.71381772e+00],
       [-8.03347803e+00, -2.37441165e+01,  3.34659182e+00],
       [-3.50615346e+00, -1.67952795e+01,  3.82607465e+00],
       [ 2.40374433e+00, -1.62445405e+01,  7.42477465e+00],
       [-5.12568326e+00, -1.61201138e+01,  2.73589574e+00],
       [-4.65226324e+00, -1.59862674e+01,  2.85080023e+00],
       [-9.25020332e+00, -1.93324169e+01, -4.03538751e-01],
       [-8.29681410e+00, -1.90429162e+01,  4.62862546e-01],
       [-3.21117454e+00, -1.96841230e+01,  5.66075405e-01],
       [-4.33178172e+00, -2.48035654e+01,  3.00889523e+00],
       [-5.53321527e+00, -3.10769064e+01,  1.37116994e+00],
       [ 1.49324118e+01, -2.61208174e+01,  2.06773833e+00],
       [ 4.25206201e+01, -1.17867382e+01,  2.31231646e-02],
       [ 6.24249873e+01,  1.05225006e+01, -8.01511704e+00],
       [ 1.71814287e+01,  1.91621151e+01, -2.84663740e+00],
       [-1.14832514e+01,  2.32617388e+01,  6.52901901e-01],
       [-3.37727354e+01,  2.44125895e+01

In [68]:
for index, row in features.iterrows():
    print index
    features[0,'features2']=row[1].reshape(-1)

0


ValueError: Length of values does not match length of index

In [247]:
features = pd.DataFrame()
bookmark=0
ft = []
for label in dic:
    for sess in dic[label]:
        features2 = pd.read_csv("vector/"+sess+'.csv',header=None)
        ft.append(features2.values.reshape(-1).T)
        #print pd.DataFrame(data=features2.values.T.reshape(-1)).transpose()
        #features.append(pd.DataFrame(features2.values.reshape(-1).transpose()),ignore_index=True)
        #features.append(pd.DataFrame(data=features2.values.T.reshape(-1)).transpose())
        #my_data = genfromtxt("vector/"+sess+'.csv', delimiter=',')
        #features2.loc[bookmark]=label,my_data
        #bookmark+=1

In [254]:
#features.set_value()
pd.DataFrame(columns=[ft[0],ft[1],ft[1],ft[2],ft[3],ft[4],ft[5],ft[6],ft[7],ft[8]])

ValueError: all arrays must be same length

In [241]:
pd.DataFrame(ft).fillna(value=0, inplace=True)
ft2=np.asarray(ft)

In [246]:
np.savetxt("file_name.csv", ft2, delimiter=",", fmt='%s', header=None)

ValueError: Expected 1D or 2D array, got 0D array instead

In [243]:
type(ft)

list

In [222]:
test = pd.DataFrame()

In [228]:
test = (pd.DataFrame(ft[0]))

In [233]:
test.append(pd.DataFrame(ft[3]))

,0
0,-6.922662
1,-23.708702
2,4.713818
3,-8.033478
4,-23.744116
5,3.346592
6,-3.506153
7,-16.795280
8,3.826075
9,2.403744


In [199]:
pd.DataFrame(np.concatenate(ft))

,0
0,-6.922662
1,-23.708702
2,4.713818
3,-8.033478
4,-23.744116
5,3.346592
6,-3.506153
7,-16.795280
8,3.826075
9,2.403744


In [187]:
from sklearn.cluster import AgglomerativeClustering

cluster = AgglomerativeClustering(n_clusters=3, affinity='euclidean', linkage='ward')  
cluster.fit_predict(np.asarray(ft).reshape(-1, 1))  

ValueError: setting an array element with a sequence.

In [156]:
from sklearn.mixture import BayesianGaussianMixture
from sklearn.mixture import GaussianMixture
gmm = GaussianMixture(n_components=3, covariance_type ='full', \
              init_params = 'random', max_iter = 100, random_state=3)
gmm.fit(ft)
print(gmm.means_.round(2))
cluster = gmm.predict(ft)

ValueError: setting an array element with a sequence.